# Using the SSC Assistant API

This Python notebook can be used on your local device using VS Code. It contains working code that you can execute locally after cloning or copying. It demonstrates how to use the SSC Assistant API using your given API token and a user Access token (oauth2).

Currently, we use the OAuth2 Client Credentials Flow. We do not support user authentication flows for API calls at this time (although it might still be technically possible). In other words, you will have to authenticate as your app itself; a user will never have to be asked for permissions to log into the app, and there will be no redirect. In order to help us support you better, we strongly recommend not using user authentication flows for the API requests at this time.

## Environment Variables

First, copy the `.env.example` that is adjacent to this file into a new file, `.env`. Then, fill in the variables according to the following instructions.

The variables can be broadly categorized as either SSC Assistant-related, or client application-related

### SSC Assistant-Related Environment Variables

#### API_ENDPOINT

The `API_ENDPOINT` points at the SSC Assistant API server that you would like to query. You will be provided this value by our team upon request.

Example:

```
API_ENDPOINT=http://127.0.0.1:5001
```

#### API_SCOPE

The `API_SCOPE` defines the permissions that your session will contain. The `applicationId` (also called the `clientId`) part is referring to the SSC Assistant's ID. You will be provided this value by our team upon request.

Example:

```
API_SCOPE=api://<applicationId>/.default
```

### Client Application-Related Identifiers

#### APP_ID

This is a credential that is specific to your application. This value can be retrieved from Azure Portal, and is generally provided by an App Administrator who has the correct permissions on Azure Portal.

#### APP_SECRET

This is a credential that is specific to your application. This value can be retrieved from Azure Portal, and is generally provided by an App Administrator who has the correct permissions on Azure Portal.

#### TENANT_ID

This is a credential that is specific to your application. This value can be retrieved from Azure Portal, and is generally provided by an App Administrator who has the correct permissions on Azure Portal.

In [ ]:
import os

import jwt
import msal
import requests
from dotenv import load_dotenv

load_dotenv()

api_endpoint = os.getenv("API_ENDPOINT")
api_scope = os.getenv("API_SCOPE")

app_id = os.getenv("APP_ID")
app_secret = os.getenv("APP_SECRET")
tenant_id = os.getenv("TENANT_ID")

app = msal.ConfidentialClientApplication(
    app_id, authority=f"https://login.microsoftonline.com/{tenant_id}",
    client_credential=app_secret)

# for confidential clients, the `.default` scope is the only scope that will work.
result = app.acquire_token_for_client(scopes=[api_scope])

if result and "access_token" in result:
    access_token = result['access_token']
    decoded_token = jwt.decode(access_token, options={"verify_signature": False})
    if "roles" in decoded_token:
        print("roles", decoded_token["roles"])
    else:
        print("No roles in token, might be missing permissions or grant approval..")
else:
    print("oops, no token", result)

## Security

API Is secured and need two different token in order to access it, first an API token (see below) and a user access token (provided by microsoft idp)

Once we have the access_token we simply make a simple call to the SSC Assistant API to ask a one-of question.

### API Token

To do so we need to ensure we have a valid user (above) and a valid token to send in the `X-API-Key` header as part of the `POST` request.

Such token can be crafted for testing as bellow:

**NOTE:** If you are not part of the dev team and you require a real token please contact the SSC Assistant team.

### Confidential Client Application access token

Normally the api relies on User access token, in this case for having 3rd parties access the API they (the application(s) in question) need to be granted the permission to request that scope.

Example of an `az cli` that does such thing. Note that the `api-permission` here is from the appRoles and not the api scopes as oposed to scopes created for user flow.

```bash
az ad app permission add --id <your-app-id> --api <api-app-id> --api-permission <scope-uuid>=Role
az ad app permission grant --id <your-app-id> --api <api-app-id> --scope api.access.app
```

### Extra documentation for the setup of the application scope

* https://stackoverflow.com/questions/77552241/getting-token-but-not-scope-inside-that-token-using-msal-code
* https://learn.microsoft.com/en-us/entra/identity-platform/quickstart-configure-app-access-web-apis
* See this [documentation here](../docs/application-config.md#granting-application-the-role)

In [29]:
import jwt
import json

api_token = jwt.encode({'roles': ['suggest',]}, 'secret', algorithm='HS256')

question = {
  "query": "Who is the president of SSC?",
  "lang": "en",
  "dedupe_citations": 1,
  "remove_markdown": 1,
}

response = requests.post(str(api_endpoint) + "/api/1.0/suggest", headers={'Authorization': f'Bearer {access_token}', 'X-API-Key': api_token}, json=question)

if response.status_code == 200:
    print(json.dumps(response.json(), indent=4))
else:
    print("Error: ", response.text)

{
    "completion_tokens": 52,
    "message": {
        "attachments": null,
        "content": "The current President of Shared Services Canada (SSC) is Scott Jones, who was appointed to this position effective September 11, 2023 .",
        "context": {
            "citations": [
                {
                    "chunk_id": null,
                    "content": "Scott Jones Scott Jones was appointed President of Shared Services Canada (SSC) effective September 11, 2023. Previous to this appointment, he was Executive Vice-President of SSC and, concurrently, Associate Deputy Minister of Public Services and Procurement Canada. Prior to April 2022, Scott was the Federal Lead on Proof of Vaccine Credentials and Associate Deputy Minister of Immigration, Refugees and Citizenship Canada (IRCC). He collaborated with federal partners, provinces and territories to successfully deliver the Canadian COVID-19 proof of vaccination program that is used for travel in Canada and internationally. S